# Installations

In [2]:
!pip3 install tensorflow
!pip3 install gym
!pip3 install keras
!pip3 install keras-rl2

  Using cached tensorflow-2.16.1-cp311-cp311-win_amd64.whl.metadata (3.5 kB)
  Using cached tensorflow_intel-2.16.1-cp311-cp311-win_amd64.whl.metadata (5.0 kB)
Using cached tensorflow-2.16.1-cp311-cp311-win_amd64.whl (2.1 kB)
Using cached tensorflow_intel-2.16.1-cp311-cp311-win_amd64.whl (377.0 MB)


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\adibo\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\tensorflow\\include\\external\\com_github_grpc_grpc\\src\\core\\ext\\filters\\client_channel\\lb_policy\\grpclb\\client_load_reporting_filter.h'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths



  Using cached keras_rl2-1.0.5-py3-none-any.whl.metadata (304 bytes)
  Using cached tensorflow-2.16.1-cp311-cp311-win_amd64.whl.metadata (3.5 kB)
  Using cached tensorflow_intel-2.16.1-cp311-cp311-win_amd64.whl.metadata (5.0 kB)
Using cached keras_rl2-1.0.5-py3-none-any.whl (52 kB)
Using cached tensorflow-2.16.1-cp311-cp311-win_amd64.whl (2.1 kB)
Using cached tensorflow_intel-2.16.1-cp311-cp311-win_amd64.whl (377.0 MB)


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\adibo\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\tensorflow\\include\\external\\com_github_grpc_grpc\\src\\core\\ext\\filters\\client_channel\\lb_policy\\grpclb\\client_load_reporting_filter.h'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths



In [3]:
#image rec
!pip3 install easyocr
!pip3 install torch torchvision torchaudio

In [4]:
!pip3 install stable-baselines3
!pip3 install shimmy

# Imports

In [1]:
import gym
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random
import time
import easyocr

In [2]:
import main

In [3]:
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# Get Stats

In [5]:
#get the score 
def getScore():
    main.screenshotScore()
    img='score.png'
    reader=easyocr.Reader(['en'], gpu=False)
    result=reader.readtext(img)
    if len(result)==0:
        return 0
    score=result[0][1]
    return int(score)

#of lines
def getLine():
    main.screenshotLine()
    img='Line.png'
    reader=easyocr.Reader(['en'], gpu=False)
    result=reader.readtext(img)
    if len(result)==0:
        return 0
    line=result[0][1]
    return int(line)

#of Blocks
def getBlocks():
    main.screenshotBlock()
    img='Block.png'
    reader=easyocr.Reader(['en'], gpu=False)
    result=reader.readtext(img)
    if len(result)==0:
        return 0
    block=result[0][1]
    return int(block)

# Tetris Environment

In [7]:
#Tettris Env
#V1 has all functions but hold
#[J_BLOCK, L_BLOCK, O_BLOCK, T_BLOCK, S_BLOCK, Z_BLOCK, L_BLOCK]
class tetrisEnv(Env):


    def __init__(self):
        self.action_space = Discrete(7)
        self.observation_space = Box(0,6, shape=(2,1), dtype= int)
        self.state= main.Curr_Next()

    def step(self, action):
        
        #update state
        next=main.getBlock()
        if (self.state[1] == next) == False:
            self.state[0]=self.state[1]
            self.state[1]=next

        #apply actions
        if action ==0:
            main.rotateCC()
        elif action ==1:
            main.rotateCW()
        elif action ==2:
            main.softDrop()
        elif action ==3:
            main.right()
        elif action ==4:
            main.left()
        elif action ==5:
            main.hardDrop()
        elif action ==6:
            main.wait()


        #check if done
        if (main.checkDone()):
            done=True
            reward= (getLine()*100) - 100
        else:
            reward=1
            done=False

        #set placeholder
        info={}

        #return stuff
        return self.state, reward, done, info

        
    def render(self):
        pass

    def reset(self):
        main.newGame()
        self.state= main.Curr_Next()
        time.sleep(2.5)
        return self.state

In [8]:
env= tetrisEnv()

# Test Environment

In [12]:
episodes=1
for i in range(episodes):
    done=False
    score=0   
    state=env.reset()
    while not done:
        action= env.action_space.sample()
        state, reward, done, info = env.step(action)
        score+=reward
    print("Episode ", end='')
    print(str(i+1)+': ', end='')
    print(score)

Using CPU. Note: This module is much faster with a GPU.


Episode 1: -27


# Reinforcment learning

In [7]:
log_path = os.path.join('Training', 'Logs')

In [15]:

model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [9]:
#train model
model.learn(total_timesteps=10000)

NameError: name 'model' is not defined

# Save and Load

In [22]:
#get path
save_Path = os.path.join('Training', 'Saved Models', 'PPO_Model_Tetris')

In [23]:
#save model
model.save(save_Path)

In [24]:
#delete model
del model

In [25]:
#load model
model = PPO.load(save_Path, env=env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


# Test Model

In [33]:
episodes=1
for i in range(episodes):
    done=False
    score=0   
    obs=env.reset()
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score+=reward
    print(score)

ValueError: Error: Unexpected observation shape (2, 1) for Box environment, please use (2, 7) or (n_env, 2, 7) for the observation shape.